# Import Libraried and packages

In [2]:
"""
from google.colab import drive
drive.mount('/content/drive')


volatility_folder = "/content/drive/MyDrive/fyp/Final report/4. Backtesting/Volatility_result/"
backtest_summary_folder = volatility_folder + "backtest_summary/"
record_folder = volatility_folder + "trade_record/"
user_test_folder = volatility_folder + "user_test/"
model_folder = "/content/drive/MyDrive/fyp/Final report/4. Backtesting/XGBoost/
""""


SyntaxError: ignored

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import floor
import pickle

import torch.nn as nn
import os.path as osp
import torch.utils.data as data
import torch.nn.functional as F

from tqdm import tqdm
import time, matplotlib, torch, os
from statsmodels.tsa.arima_model import ARIMA
from torch.utils.tensorboard import SummaryWriter

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
import os
from os import walk

import plotly
%matplotlib inline
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from datetime import datetime
#import seaborn as sns

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [6]:
#For DNN / CNN+LSTM
import keras.backend as K
from keras.models import Sequential
from keras.layers import Conv1D, LSTM, Flatten, Dense

In [7]:
!pip install -q stockstats

# Load stock data from API

In [8]:
import pandas_datareader.data as web
from stockstats import StockDataFrame

#Setting the index values as timestamp
def process(code):
  print("Enter the code of the model that you want to implement")
  #code = input()
  #final_code = '^'+code
  final_code = code
  start_date = "2012-01-01"
  end_date = "2021-12-31"
  stock = web.DataReader(final_code , 'stooq',start=start_date, end=end_date)
  stock.columns = ["open","high","low","close","volume"]
  
  # Use online package to generate additional features
  x = StockDataFrame(stock)
  stock = x[['open','high','low','close','volume',
          'boll', 'boll_ub', 'boll_lb',
          'macd', 'macdh', 'macds',
          'rsi_11', 'rsi_14', 'rsi_21']]
  stock.index = [int(str(ind)[0:4]+str(ind)[5:7]+str(ind)[8:10]) for ind in stock.index]
  stock = stock.sort_index()

  return stock

In [9]:
stock = process("IBM")

Enter the code of the model that you want to implement


In [10]:
stock

,open,high,low,close,volume,boll,boll_ub,boll_lb,macd,macdh,macds,rsi_11,rsi_14,rsi_21
20120103,128.88,130.23,128.38,128.58,8180058,128.1790,134.932612,121.425388,-1.811274,0.489102,-2.300376,51.281366,47.190995,42.920670
20120104,128.10,128.60,127.63,128.07,6297526,128.3965,135.476771,121.316229,-2.091832,0.330819,-2.422652,48.488535,44.808561,41.192557
20120105,127.56,127.71,126.38,127.46,6466200,128.6025,135.879775,121.325225,-2.367574,0.137783,-2.505356,45.064237,41.897279,39.091982
20120106,127.28,127.34,125.83,125.98,7094969,128.9115,136.502568,121.320432,-2.622973,-0.083171,-2.539802,35.625325,34.061158,33.612565
20120109,125.76,125.80,124.42,125.33,7535572,129.2665,136.943913,121.589087,-2.760113,-0.241104,-2.519009,30.883945,30.223364,31.016798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20211227,129.08,130.08,128.40,130.05,4346229,131.4500,133.282321,129.617679,-0.099397,-0.058552,-0.040846,8.180361,8.545592,8.997984
20211228,130.03,131.63,130.02,131.05,3488601,131.8000,132.899939,130.700061,-0.042801,-0.026749,-0.016052,14.491914,14.911940,15.424720
20211229,130.82,132.61,130.73,131.76,4291007,132.0500,132.611070,131.488930,-0.010219,-0.007870,-0.002349,28.868360,29.305681,29.832936
20211230,132.16,132.77,131.74,132.32,3196152,132.1950,132.548553,131.841447,0.005609,0.002493,0.003116,100.000000,100.000000,100.000000


# Data preprocessing for regression models

In [11]:
from datetime import datetime
def standard_date_format(date_in_num):
    date = str(date_in_num)
    return datetime.strptime(date,'%Y%m%d').strftime('%Y-%m-%d')

In [12]:
def convert_prediction(X,y,pred):

    print("a")
    col_list =[] 
    for col in X.columns:
        if "open" in col:
            col_list.append(col)
    print("b")
    
    mean_5_day = np.mean(X.loc[:,col_list],axis=1)
    mean_5_day.index = y.index
    std_5_day = np.std(X.loc[:,col_list],axis=1)
    std_5_day.index = y.index
    a = pd.DataFrame(index = y.index,columns=['mean','std','pred'])
    a.loc[:,"mean"] = mean_5_day
    a.loc[:,"std"] = std_5_day
    a.loc[:,"pred"] = pred
    a.loc[:,"converted"] = pred*std_5_day+mean_5_day
    return a["converted"]

In [13]:
def convert_decision(stock_,pred):
    h = np.array(pred)
    action = []
    status = "N"
    five_sma = stock_['open'].rolling(window=5,min_periods=1).mean()

    ind_sma = five_sma.index
    ind_pred = pred.index
    union_ind = list(ind_sma.intersection(ind_pred))

    five_sma = five_sma.filter(items = union_ind)
    pred = pred.filter(items = union_ind)

    for i in range(len(union_ind)):
      
        if pred.iloc[i] >= five_sma.iloc[i]:
            if status == "N":
                action.append("Buy")
                status = "Buy"
            else:
                action.append("Hold")
        else:
            if status == "Buy":
                action.append("Sell")
                status = "N"
            else:
                action.append("Hold")

    result = (pd.DataFrame(action,
                         columns=["Action"], index = pred.index))
                
    return result

In [14]:
# Split the train and test data
def custom_split(data,start,end):
    train = (data.index >= start) & (data.index <= end)
    train_X = data[train]
    
    return train_X

 Split the train and test data

In [15]:
train_X = custom_split(stock,start = 20130101,end = 20171031)
valid_X = custom_split(stock,start = 20171101,end = 20181231)
test_X = custom_split(stock,start = 20190101,end = 20201231)

Label the target result (opening price on 11th day)

In [16]:
# Assume we use 10 days price data to predict closing price of the 11th day
num_day_to_predict = 5

In [17]:
def produce_result_target_price(X,num_day = 5,result_col_name = "result_price"):
    y = pd.DataFrame(np.nan, index=X.index, columns=[result_col_name])
    for i in range(5,len(X)):
        y.loc[X.index[i-1],result_col_name] = X.loc[X.index[i],"open"]
    
    return y

In [18]:
train_y = produce_result_target_price(train_X,num_day_to_predict)
valid_y = produce_result_target_price(valid_X,num_day_to_predict)
test_y = produce_result_target_price(test_X,num_day_to_predict)

Transform the 10-day data into one vector

In [19]:
def transform_x_data_to_one_vector(X,num_day):
    col_name = []
    for i in range(num_day):
        for j in X.columns:
            col_name.append(j+"-"+str(i))
    new_X = pd.DataFrame(np.nan, index=X.index, columns=col_name)
    
    for i in range(num_day-1,len(X)):
        for col in col_name:
            split_list = col.split("-")
            new_X.loc[X.index[i],col] = X.loc[X.index[i-int(split_list[1])],split_list[0]]
    
    return new_X

In [20]:
train_X_5 = transform_x_data_to_one_vector(train_X,num_day_to_predict)
valid_X_5 = transform_x_data_to_one_vector(valid_X,num_day_to_predict)
test_X_5 = transform_x_data_to_one_vector(test_X,num_day_to_predict)

 Drop out rows with NaN

In [21]:
def drop_nan_row_y(y,num_day = 5):
    drop_list = [y.index[i] for i in range(num_day-1)]
    drop_list.append(y.index[-1])
    return y.drop(drop_list)

def drop_nan_row_X_5(X,num_day = 5):
    drop_list = [X.index[i] for i in range(num_day-1)]
    drop_list.append(X.index[-1])
    return X.drop(drop_list)

In [22]:
new_train_X_5 = drop_nan_row_X_5(train_X_5,num_day_to_predict)
new_train_y = drop_nan_row_y(train_y,num_day_to_predict)

new_valid_X_5  = drop_nan_row_X_5(valid_X_5,num_day_to_predict)
new_valid_y = drop_nan_row_y(valid_y,num_day_to_predict)

new_test_X_5  = drop_nan_row_X_5(test_X_5,num_day_to_predict)
new_test_y = drop_nan_row_y(test_y,num_day_to_predict)

In [23]:
# Keep a copy for reverting prediction result
old_train_X_5 = new_train_X_5
old_train_y = new_train_y
old_valid_X_5 = new_valid_X_5
old_valid_y = new_valid_y
old_test_X_5 = new_test_X_5
old_test_y = new_test_y

In [24]:
### Normalize data row by row

In [25]:
def normalize_data_by_row(X,y):
    norm_X = X.sub(X.mean(axis=0), axis=1).div(X.std(axis=0), axis=1)
    
    col_list =[] 
    for col in X.columns:
        if "open" in col:
            col_list.append(col)
    
    temp_y = y.loc[:,"result_price"]
    mean_5_day = np.mean(X.loc[:,col_list],axis=1)
    mean_5_day.index = temp_y.index
    std_5_day = np.std(X.loc[:,col_list],axis=1)
    std_5_day.index = temp_y.index
    

    norm_y = pd.DataFrame((temp_y-mean_5_day)/std_5_day,
                          columns=["result_price"])
                         
    return (norm_X,norm_y)

In [26]:
new_train_X_5,new_train_y = normalize_data_by_row(new_train_X_5,new_train_y)
new_valid_X_5,new_valid_y = normalize_data_by_row(new_valid_X_5,new_valid_y)
new_test_X_5,new_test_y = normalize_data_by_row(new_test_X_5,new_test_y)

In [27]:
new_train_X_5 = drop_nan_row_X_5(new_train_X_5,num_day_to_predict)
new_valid_X_5 = drop_nan_row_X_5(new_valid_X_5,num_day_to_predict)
new_test_X_5 = drop_nan_row_X_5(new_test_X_5,num_day_to_predict)

In [28]:
new_train_y = drop_nan_row_y(new_train_y,num_day_to_predict)
new_valid_y = drop_nan_row_y(new_valid_y,num_day_to_predict)
new_test_y = drop_nan_row_y(new_test_y,num_day_to_predict)

In [29]:
len(new_test_X_5.columns)

70

# XGBboost Training

In [30]:
from xgboost import XGBRegressor
model_xgb = XGBRegressor(learning_rate = 0.01,n_estimators=1000)
model_xgb.fit(new_train_X_5, new_train_y, early_stopping_rounds=5, eval_set=[(new_valid_X_5, new_valid_y)], verbose=False)
xgb_pred_y = model_xgb.predict(new_test_X_5)

[07:30:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [31]:
#Performance evaluation
print('XGBoost PERFORMANCE')
print('r2 score: '+str(r2_score(new_test_y, xgb_pred_y)))
print('RMSE : '+str(np.sqrt(mean_squared_error(new_test_y, xgb_pred_y))))
print("Mean Absolute Error : " + str(mean_absolute_error(new_test_y,xgb_pred_y)))

XGBoost PERFORMANCE
r2 score: 0.39090347765863553
RMSE : 1.7580605387765824
Mean Absolute Error : 1.1670200978605854


In [33]:
from sklearn.metrics import classification_report, confusion_matrix

In [34]:
print(str(classification_report(new_test_y, xgb_pred_y)))
print("\n\n")
print(str(confusion_matrix(new_test_y, xgb_pred_y)))

ValueError: ignored

In [ ]:
#with open("Xgb.pickle", "wb") as f:
    #pickle.dump(model_xgb, f)